# Installing the necessary libraries and downloading the datataset
I used the SQuAD (Stanford question answering dataset) 

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 85.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 92.2 MB/s eta 0:00:00


In [2]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 46.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 20.2 MB/s eta 0:00:00


Exploring the dataset

In [5]:
from datasets import load_dataset



In [6]:
data = load_dataset("squad")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Dataset squad downloaded and prepared to /root/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
data['train'][0]

{'id': '5733be284776f41900661182',
 'title': 'University_of_Notre_Dame',
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}

#Preprocessing

Loading the Tokenizer for preprocessing

In [8]:
from transformers import AutoTokenizer

In [9]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

In [10]:
q = data['train'][0]['question']
c = data['train'][0]['context']

In [11]:
input1 = tokenizer(q, c)

In [12]:
tokenizer.decode(input1['input_ids'])

'[CLS] To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France? [SEP] Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend " Venite Ad Me Omnes ". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive ( and in a direct line that connects through 3 statues and the Gold Dome ), is a simple, modern stone statue of Mary. [SEP]'

In [17]:
max_length = 384
stride = 128

<h2>Function for prerocessing the traing data</h2>

Indexing the answers in the context </br>
Splitting the context if it's to long

In [19]:
def preprocess_train(inp):
  questions = [q.strip() for q in inp["question"]]
  
  inputs = tokenizer(
    questions,
    inp["context"],
    max_length=max_length,
    truncation="only_second",
    stride=stride,
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    padding="max_length",
  )
  
  offset_mapping = inputs.pop("offset_mapping")
  sample_map = inputs.pop("overflow_to_sample_mapping")
  answers = inp["answers"]
  start_ind = []
  end_ind = []
  
  for i, offset in enumerate(offset_mapping):
    sample_idx = sample_map[i]
    answer = answers[sample_idx]
    start_char = answer['answer_start'][0]
    end_char = answer['answer_start'][0] + len(answer["text"][0])
    sequence_ids = inputs.sequence_ids(i)
    idx = 0
    while sequence_ids[idx] != 1:
      idx += 1
    context_start = idx
    while sequence_ids[idx] == 1:
      idx += 1
    context_end = idx - 1

    if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
      start_ind.append(0)
      end_ind.append(0)
    else:
      idx = context_start
      while idx <= context_end and offset[idx][0] <= start_char:
        idx += 1
      start_ind.append(idx - 1)

      idx = context_end
      while idx >= context_start and offset[idx][1] >= end_char:
        idx -= 1
      end_ind.append(idx + 1) 
    
  inputs['start_positions'] = start_ind

  inputs['end_positions'] = end_ind
  return inputs

In [20]:
train_dataset = data['train'].select(range(5000)).map(preprocess_train,
                  batched=True,
    remove_columns=data["train"].column_names,)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [21]:
len(train_dataset), len(data['train'])

(5103, 87599)

Preprocessing the validation dataset

In [22]:
def preprocess_validation(inp):
  questions = [q.strip() for q in inp["question"]]

  inputs = tokenizer(
    questions,
    inp["context"],
    max_length=max_length,
    truncation="only_second",
    stride=stride,
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    padding="max_length",
  )
  
  sample_map = inputs.pop("overflow_to_sample_mapping")
  example_ids = []  
  
  for i in range(len(inputs["input_ids"])):
    sample_idx = sample_map[i]
    example_ids.append(inp["id"][sample_idx])

    sequence_ids = inputs.sequence_ids(i)
    offset = inputs["offset_mapping"][i]
    inputs["offset_mapping"][i] = [
      o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
    ]

  inputs["example_id"] = example_ids

  return inputs
  

In [23]:
validation_dataset = data["validation"].map(
    preprocess_validation,
    batched=True,
    remove_columns=data["validation"].column_names,
)
len(data["validation"]), len(validation_dataset)

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

(10570, 10822)

In [24]:
from transformers import TFAutoModelForQuestionAnswering
import tensorflow as tf

In [25]:
model = TFAutoModelForQuestionAnswering.from_pretrained('bert-base-cased')

All model checkpoint layers were used when initializing TFBertForQuestionAnswering.

Some layers of TFBertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['qa_outputs']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
from huggingface_hub import notebook_login

In [27]:
notebook_login()

In [78]:
from transformers import DefaultDataCollator
data_collator = DefaultDataCollator(return_tensors="tf")


In [79]:
final_train_dataset = model.prepare_tf_dataset(
    train_dataset,
    collate_fn=data_collator,
    shuffle=True,
    batch_size=16
)

final_validation_dataset = model.prepare_tf_dataset(
    validation_dataset,
    collate_fn=data_collator,
    shuffle=True,
    batch_size=16
)

Creating the model and setting up the parameters

In [80]:
from transformers import create_optimizer
from transformers.keras_callbacks import PushToHubCallback


In [81]:
epochs = 3
steps = len(final_train_dataset) * epochs
optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=0,
    num_train_steps=steps,
    weight_decay_rate=0.01,
)
model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [82]:
tf.keras.mixed_precision.set_global_policy("mixed_float16")

In [83]:
from transformers.keras_callbacks import PushToHubCallback

callback = PushToHubCallback(output_dir="bert-finetuned-squad2", tokenizer=tokenizer)


/content/bert-finetuned-squad2 is already a clone of https://huggingface.co/zsomai/bert-finetuned-squad2. Make sure you pull the latest changes with `repo.git_pull()`.


In [84]:
small_validation = data["validation"].select(range(100))
trained_checkpoint = "distilbert-base-cased-distilled-squad"

tokenizer = AutoTokenizer.from_pretrained(trained_checkpoint)
eval_set = small_validation.map(
    preprocess_validation,
    batched=True,
    remove_columns=data["validation"].column_names,
)

In [85]:
model.fit(final_train_dataset, callbacks=[callback], epochs=epochs)

Epoch 1/3
318/318 [==============================] - 464s 1s/step - loss: 0.7777
Epoch 2/3
318/318 [==============================] - 459s 1s/step - loss: 0.4589
Epoch 3/3
318/318 [==============================] - 460s 1s/step - loss: 0.3174


#Testing the model

In [86]:
from transformers import pipeline

In [87]:
question_answerer = pipeline('question-answering', model='zsomai/bert-finetuned-squad2')

All model checkpoint layers were used when initializing TFBertForQuestionAnswering.

All the layers of TFBertForQuestionAnswering were initialized from the model checkpoint at zsomai/bert-finetuned-squad2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForQuestionAnswering for predictions without further training.


In [93]:
id = 4
question_answerer(question=data["validation"][id]['question'], context=data["validation"][id]['context'])

{'score': 0.09124755859375,
 'start': 630,
 'end': 644,
 'answer': 'Roman numerals'}

In [100]:
id = 4

In [101]:
data["validation"][id]['question']

'What color was used to emphasize the 50th anniversary of the Super Bowl?'

In [99]:
data["validation"][id]['answers']

{'text': ['"golden anniversary"', 'gold-themed', '"golden anniversary'],
 'answer_start': [487, 521, 487]}

In [92]:
data["validation"][id]['context']

'Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi\'s Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.'

In [61]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [62]:
y_validate = [{"id": question['id'], 'answers': question['answers']} for question in small_validation]

In [ ]:
y_validate

In [ ]:
y_predicted = []
for question in small_validation:
  resp = question_answerer(question=question['question'], context=question['context'])
  y_predicted.append({"id": question['id'], 'prediction_text':resp['answer']})

In [74]:
y_validate = y_validate[:145]

In [ ]:
y_validate

In [72]:
len(y_predicted)

145

In [76]:
import evaluate

metric = evaluate.load("squad")

In [77]:
metric.compute(predictions=y_predicted, references=y_validate)

{'exact_match': 73.10344827586206, 'f1': 79.4433497536946}